In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_countries,check_doi_glutton,normalize
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [27]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [32]:
pd.Series([list(pd.Series([x[i].get('wg') for i in range(len(x))]).drop_duplicates()) for x in list(df_ipcc.drop_duplicates(subset=['doi']).dropna(subset=['doi']).wg_chap)]).value_counts()

[2]                   22965
[3]                   12402
[1]                   10579
[2_cross]              3664
[2, 2_cross]           1561
[1, 2]                 1473
[2, 3]                 1227
[1, 2, 2_cross]         252
[1, 3]                  189
[1, 2_cross]            178
[1, 2, 3]               172
[2, 2_cross, 3]          95
[2_cross, 3]             34
[1, 2, 2_cross, 3]       24
[1, 2_cross, 3]           6
Name: count, dtype: int64

In [21]:
df_ipcc['year']=df_ipcc.loc[:,'year'].apply(lambda x: get_year_ipcc(x))

In [26]:
min([int(x) for x in list(df_ipcc.year_OA.drop_duplicates().dropna())])

1686

In [4]:
df_ipcc['test_glutton']=df_ipcc.apply(lambda row: check_doi_glutton(row), axis=1)

In [7]:
df_ipcc.test_glutton.value_counts()

test_glutton
True     63904
False    17311
Name: count, dtype: int64

In [6]:
df_ipcc[df_ipcc.test_glutton==False]

,year,url,doi,title,author,freq,wg,chap,countries,concepts,sdg,topics,is_OA_available,title_OA,year_OA,wg_chap,test_glutton
0,2019,https://www.frontiersin.org/research-topics/82...,10.3389/978-2-88963-118-6,{OceanObs'19: An Ocean of Opportunity. Volume 1},None,1,[wg1],[wg1_chap_01],[None],"[{'name': 'Volume (thermodynamics)'}, {'name':...","[{'id': '14', 'name': 'Life below water'}]",[{'name': 'Marine Biodiversity and Ecosystem F...,1.0,Oceanobs'19: An Ocean of Opportunity. Volume I,2020.0,"[{'name': 'wg1_chap_01', 'wg': '1', 'chap': 1}]",False
1,2000,None,10.1007/978-94-015-9612-1,"{Eduard Br{\""{u}}ckner – The Sources and Conse...",None,1,[wg1],[wg1_chap_01],[None],"[{'name': 'Geographer'}, {'name': 'Climate cha...","[{'id': '13', 'name': 'Climate action'}]",[{'name': 'Climate Change and Paleoclimatology...,1.0,Eduard Brückner — The Sources and Consequences...,2000.0,"[{'name': 'wg1_chap_01', 'wg': '1', 'chap': 1}]",False
2,1978,http://pure.iiasa.ac.at/id/eprint/821/1/XB-78-...,None,"{Carbon Dioxide, Climate and Society: Proceedi...",None,1,None,None,[None],None,None,None,NaN,None,NaN,None,False
3,2017,None,None,{Cultures of Prediction in Atmospheric and Cli...,None,1,None,None,[None],None,None,None,NaN,None,NaN,None,False
4,1900,None,10.1002/andp.19003081208,"{{\""{U}}ber die Bedeutung des Wasserdampfes un...","{\AA}ngstr{\""{o}}m, Knut",1,"[wg1, wg1]","[wg1_chap_01, wg1_chap_07]",[US],"[{'name': 'Physics'}, {'name': 'Humanities'}, ...",None,[{'name': 'Stratospheric Chemistry and Climate...,1.0,Ueber die Bedeutung des Wasserdampfes und der ...,1900.0,"[{'name': 'wg1_chap_01', 'wg': '1', 'chap': 1}...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81162,2018.0,https://doi.org/10.1080%2F09640568.2018.1513832,10.1080/09640568.2018.1513832,Special section: advancing the role of cities ...,Jeroen van der Heijden and James Patterson and...,1,"[wg2, wg3]","[wg2_chap_06, wg3_chap_17]","[FI, NL, AU, KR, NZ]","[{'name': 'Corporate governance'}, {'name': 'P...","[{'id': '13', 'name': 'Climate action'}]",[{'name': 'Sustainability Transitions and Resi...,1.0,Special section: advancing the role of cities ...,2018.0,"[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6}...",False
81163,2018.0,https://doi.org/10.1016%2Fj.cosust.2018.03.006,10.1016/j.cosust.2018.03.006,The limits of voluntary programs for low-carbo...,Jeroen Van der Heijden,1,[wg3],[wg3_chap_17],"[NL, AU]","[{'name': 'Turnover'}, {'name': 'Resource cons...","[{'id': '13', 'name': 'Climate action'}]",[{'name': 'Sustainability Transitions and Resi...,1.0,The limits of voluntary programs for low-carbo...,2018.0,"[{'name': 'wg3_chap_17', 'wg': '3', 'chap': 17}]",False
81173,2000.0,https://doi.org/10.1080%2F00071773.2000.11007303,10.1080/00071773.2000.11007303,$\less$b$\greater$The View from Within: First ...,N.E. Wetherick,1,[wg3],[wg3_chap_17],[None],"[{'name': 'Consciousness'}, {'name': 'Philosop...","[{'id': '10', 'name': 'Reduced inequalities'}]",[{'name': 'Physics of the Mind and Cognitive S...,1.0,<b>The View from Within: First Person Approach...,2000.0,"[{'name': 'wg3_chap_17', 'wg': '3', 'chap': 17}]",False
81191,2018.0,https://doi.org/10.17645%2Fup.v3i2.1245,10.17645/up.v3i2.1245,{WGV}: An Australian Urban Precinct Case Study...,Jason Wiktorowicz and Tanya Babaeff and Jessic...,1,"[wg2, wg3, wg3]","[wg2_chap_11, wg3_chap_08, wg3_chap_17]",[AU],"[{'name': 'Precinct'}, {'name': 'Sustainabilit...",None,"[{'name': 'Integrated Management of Water, Ene...",1.0,WGV: An Australian Urban Precinct Case Study t...,2018.0,"[{'name': 'wg2_chap_11', 'wg': '2', 'chap': 11...",False


In [8]:
normalize(df_ipcc[df_ipcc.test_glutton==False].title[81195])

'the potential contribution of disruptive low carbon innovations to 1 5 textdegree c climate mitigation'

In [9]:
normalize(df_ipcc[df_ipcc.test_glutton==False].title_OA[81195])

'the potential contribution of disruptive low carbon innovations to 1 5 ° c climate mitigation'

In [11]:
df_ipcc[df_ipcc.test_glutton==False].year[81195]

2018.0

In [10]:
df_ipcc[df_ipcc.test_glutton==False].year_OA[81195]

2018.0

In [ ]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [ ]:
df_ipcc_group=df_ipcc.groupby(by=['doi','year'], dropna=True).agg({'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [ ]:
df_ipcc_group[df_ipcc_group.freq>1]

In [ ]:
df_ipcc_group.ipcc[4]

In [ ]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [ ]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [ ]:
df_ipbes_group=df_ipbes.groupby(by=['doi','year'], dropna=True).agg({'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipbes_group=df_ipbes_group.reset_index()

In [ ]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
df_ipbes['year']=df_ipbes.loc[:,'date'].apply(lambda x: get_year(str(x)) if pd.isna(x)==False else None)

In [ ]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes

In [ ]:
update_bso_publications()

In [ ]:
bso=pd.DataFrame(get_bso_publications())

In [ ]:
df_bso=bso.transpose().reset_index()

In [ ]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [ ]:
df_bso

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left')

In [ ]:
df_ipcc_ipbes_bso['countries']=df_ipcc_ipbes_bso.apply(lambda row: get_countries(row),axis=1)

In [ ]:
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso[['doi', 'ipcc', 'ipbes', 'countries', 'rors', 'bso_local_affiliations']]

In [ ]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [ ]:
df_iso=df_iso.rename(columns={'code':'countries'})

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_iso, on='countries', how='left')
del df_ipcc_ipbes_bso['countries']
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso.rename(columns={'name':'countries'})

In [ ]:
df_brevet_doi=pd.DataFrame('brevet_doi.json')

In [ ]:
df_brevet_doi

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [ ]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

In [ ]:
df_cache

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [ ]:
dict_doi_bso=[]

In [ ]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [ ]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', lines=True)

In [ ]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [ ]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

In [ ]:
current_dict = dd.to_dict(orient='records')

In [ ]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [ ]:
df_elastic_input=pd.DataFrame(current_dict)

In [ ]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [ ]:
del df_elastic_input['openalex_abstract_inverted_index']

In [ ]:
len(df_elastic_input)

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [ ]:
#df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'